# RAG Pipeline: Evaluation and Analysis

This notebook focuses on running the complete Retrieval-Augmented Generation (RAG) pipeline on a set of representative questions. We will analyze how the system retrieves relevant complaints and generates answers using the LLM.

### Pipeline Components:
1. **Vector Store**: FAISS index containing complaint embeddings.
2. **Retriever**: Similarity search mechanism to find top-K relevant documents.
3. **LLM**: Instruction-tuned model (FLAN-T5-Small) to generate answers based on context.
4. **Prompt Engineering**: Structured templates to guide the LLM's response.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import IPython
from IPython.display import display, Markdown

# allow imports from project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src import config, vectorstore, llm
from src.rag_pipeline import RAGPipeline

config.setup_hf_cache()
print("✓ Environment setup complete")

c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[OK] HuggingFace cache set to: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf
✓ Environment setup complete


## 1. Initialize Production Pipeline

We initialize the `RAGPipeline` class, which encapsulates the loading of the vector store and the LLM engine.

In [2]:
pipeline = RAGPipeline()
if pipeline.initialize():
    print("✓ RAG Pipeline initialized and ready!")
else:
    print("❌ Pipeline initialization failed. Check your vector store and model paths.")


--- Initializing RAG Pipeline ---
Attempting to load vector store from: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
  (First run will download ~80MB to c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 142.55it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[OK] Embedding model loaded
Loading existing FAISS index from c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss...
[OK] Vector store loaded (ntotal=3)
Loading Alternative LLM: google/flan-t5-small
  Max new tokens: 512
  Temperature: 0.1


Loading weights: 100%|██████████| 190/190 [00:01<00:00, 119.20it/s, Materializing param=shared.weight]                                                       
The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'temperature', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM',

--- Pipeline Ready ---

✓ RAG Pipeline initialized and ready!


## 2. Representative Evaluation Questions

Here is a curated list of questions that represent common consumer concerns. We will use these to evaluate the system's performance.

In [3]:
eval_questions = [
    "What are common issues consumers face with credit card interest rates?",
    "How do customers describe problems with mortgage loan modifications?",
    "What are the most frequent complaints regarding unauthorized transactions on checking accounts?",
    "What reasons do consumers give for disputes regarding credit reporting inaccuracies?",
    "What are the common themes in complaints against debt collection agencies' practices?",
    "How do consumers react to unexpected fees on prepaid cards?",
    "What are the typical complaints about vehicle loan servicing?",
    "What issues do consumers report when trying to close their savings accounts?"
]

print(f"✓ Defined {len(eval_questions)} representative questions.")

✓ Defined 8 representative questions.


## 3. Running RAG Pipeline and Results Analysis

For each question, we run the full RAG cycle:
1. **Retrieve**: Find top 5 relevant complaint chunks.
2. **Augment**: Format the chunks into a context string for the prompt.
3. **Generate**: Use FLAN-T5-Small to generate a professional answer based *only* on the context.

In [4]:
all_results = []

for i, query in enumerate(eval_questions, 1):
    display(Markdown(f"### Question {i}: {query}"))
    
    # Run the pipeline
    result = pipeline.run(query, k=5)
    
    # Display the answer
    display(Markdown(f"**Generated Answer:** {result['answer']}"))
    
    # Display source metadata summary
    sources = result['source_documents']
    display(Markdown(f"*Retrieved {len(sources)} sources from products: {set([doc.metadata.get('product', 'N/A') for doc in sources])}*"))
    
    # Show a snippet of the most relevant source
    top_source = sources[0]
    display(Markdown(f"**Top Source Snippet (Complaint {top_source.metadata.get('complaint_id')}):**  \n*{top_source.page_content[:250]}...*"))
    
    display(Markdown("---"))
    
    all_results.append({
        "question": query,
        "answer": result['answer'],
        "sources": sources
    })

### Question 1: What are common issues consumers face with credit card interest rates?

Processing Query: 'What are common issues consumers face with credit card interest rates?'
Generating answer based on 3 sources...


Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**Generated Answer:** You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say "I don't have enough information".

Context:
Content: This is a complaint about credit card fees.

Content: I have an issue with my personal loan interest rate.

Content: My savings account was closed without notice.

Question: What are common issues consumers face with credit card interest rates?

Answer:

*Retrieved 3 sources from products: {'Credit card', 'Savings account', 'Personal loan'}*

...

Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**Generated Answer:** You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say "I don't have enough information".

Context:
Content: My savings account was closed without notice.

Content: This is a complaint about credit card fees.

Content: I have an issue with my personal loan interest rate.

Question: What issues do consumers report when trying to close their savings accounts?

Answer:

*Retrieved 3 sources from products: {'Savings account', 'Credit card', 'Personal loan'}*

**Top Source Snippet (Complaint 3):**  
*My savings account was closed without notice....*

---

## 4. Pipeline Evaluation Table

We consolidate the results into a structured evaluation table. This can be exported to your report. 

> **Note:** The `Quality Score` and `Comments/Analysis` columns are intended for manual review to ensure the AI's answers are grounded and accurate.

In [5]:
evaluation_data = []

for res in all_results:
    # Get 1-2 source IDs
    source_ids = [doc.metadata.get('complaint_id', 'N/A') for doc in res['sources'][:2]]
    
    evaluation_data.append({
        "Question": res['question'],
        "Generated Answer": res['answer'],
        "Retrieved Sources": ", ".join(map(str, source_ids)),
        "Quality Score (1-5)": "",  # Placeholder for manual review
        "Comments/Analysis": ""      # Placeholder for manual review
    })

df_eval = pd.DataFrame(evaluation_data)

# Set column display for better readability in the notebook
pd.set_option('display.max_colwidth', 300)
display(df_eval)



,Question,Generated Answer,Retrieved Sources,Quality Score (1-5),Comments/Analysis
0,What are common issues consumers face with credit card interest rates?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: This is a complaint about credit card fees.\n\nContent: I have an issue with my personal loan interest...","1, 2",,
1,How do customers describe problems with mortgage loan modifications?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: I have an issue with my personal loan interest rate.\n\nContent: This is a complaint about credit card...","2, 1",,
2,What are the most frequent complaints regarding unauthorized transactions on checking accounts?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: This is a complaint about credit card fees.\n\nContent: My savings account was closed without notice.\...","1, 3",,
3,What reasons do consumers give for disputes regarding credit reporting inaccuracies?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: This is a complaint about credit card fees.\n\nContent: I have an issue with my personal loan interest...","1, 2",,
4,What are the common themes in complaints against debt collection agencies' practices?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: This is a complaint about credit card fees.\n\nContent: I have an issue with my personal loan interest...","1, 2",,
5,How do consumers react to unexpected fees on prepaid cards?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: This is a complaint about credit card fees.\n\nContent: My savings account was closed without notice.\...","1, 3",,
6,What are the typical complaints about vehicle loan servicing?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: I have an issue with my personal loan interest rate.\n\nContent: This is a complaint about credit card...","2, 1",,
7,What issues do consumers report when trying to close their savings accounts?,"You are a helpful financial assistant. Answer the question using ONLY the following context. If you cannot answer using the context, say ""I don't have enough information"".\n\nContext:\nContent: My savings account was closed without notice.\n\nContent: This is a complaint about credit card fees.\...","3, 1",,
